# Unsupervised discretization
Dataset: iris

Updated at: 28 June 22

By: Sam

### About Dataset
Number of Instances: 150 (50 in each of three classes)
Number of Attributes: 4 numeric, predictive attributes and the class
Attribute Information:
   1. sepal length in cm
   2. sepal width in cm
   3. petal length in cm
   4. petal width in cm
   5. class: 
      -- Iris Setosa
      -- Iris Versicolour
      -- Iris Virginica

Missing Attribute Values: None

In [1]:
# Load library
import pandas as pd
import numpy as np
import time

In [2]:
from sklearn.preprocessing import KBinsDiscretizer as kbins # also use for unsupervised

In [3]:
from feature_engine.discretisation import EqualFrequencyDiscretiser as efd
from feature_engine.discretisation import EqualWidthDiscretiser as ewd

In [4]:
# Load dataset
data = pd.read_csv('clean_iris.csv')

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   slength  150 non-null    float64
 1   swidth   150 non-null    float64
 2   plength  150 non-null    float64
 3   pwidth   150 non-null    float64
 4   label    150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [6]:
# get list of numeric attributes to discretize
num_col = data.select_dtypes(include=np.number).columns
num_col = num_col.tolist()

In [39]:
# Encode label (for training and evaluation later, cannot use string title)
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
data['label'] = le.fit_transform(data['label'])

In [40]:
data.info

<bound method DataFrame.info of      slength  swidth  plength  pwidth  label
0        5.1     3.5      1.4     0.2      0
1        4.9     3.0      1.4     0.2      0
2        4.7     3.2      1.3     0.2      0
3        4.6     3.1      1.5     0.2      0
4        5.0     3.6      1.4     0.2      0
..       ...     ...      ...     ...    ...
145      6.7     3.0      5.2     2.3      2
146      6.3     2.5      5.0     1.9      2
147      6.5     3.0      5.2     2.0      2
148      6.2     3.4      5.4     2.3      2
149      5.9     3.0      5.1     1.8      2

[150 rows x 5 columns]>

## Equal Width Discretization

In [41]:
# Define function: Inputs: dataset, number of parameters

def ewd_disc(data, k):
    ## set up the discretisation transformer
    ewd_disc = ewd(bins=k, variables=num_col, return_boundaries=False)
    '''
    Parameters
    ----------
    bins : int, default=10
        Desired number of equal width intervals / bins.

    variables : list
        The list of numerical variables to transform. If None, the
        discretiser will automatically select all numerical type variables.

    return_object : bool, default=False
        Whether the numbers in the discrete variable should be returned as
        numeric or as object. The decision should be made by the user based on
        whether they would like to proceed the engineering of the variable as
        if it was numerical or categorical.

    return_boundaries: bool, default=False
        whether the output should be the interval boundaries. If True, it returns
        the interval boundaries. If False, it returns integers.
    '''
    ## fit the transformer
    ewd_disc.fit(data)
    ## transform the data
    data_ewd = ewd_disc.transform(data)
    ## binner_dict contains the boundaries of the different bins: 
    # stores the interval limits identified for each variable
    ewd_disc.binner_dict_
    return data_ewd  # return dataset after discretization

### EWD - Scenario 1: k = 4

In [42]:
# Perform discretization
k = 4
start = time.time() # Starting  time
data_ewd1 = ewd_disc(data, k)
end = time.time()
ewd_t = end - start
print("Discretization time, EWD, k = ", k,":",ewd_t) # Total time execution

Discretization time, EWD, k =  4 : 0.016412019729614258


In [43]:
# OUTPUT:
data_ewd1.head()

,slength,swidth,plength,pwidth,label
0,0,2,0,0,0
1,0,1,0,0,0
2,0,1,0,0,0
3,0,1,0,0,0
4,0,2,0,0,0


In [44]:
## OUTPUT: Check number of instance in each interval in the data_ewd
# With equal width discretisation, each bin does not necessarily 
# contain the same number of observations.
for col in num_col:
    print(col)
    print(data_ewd1.groupby(col)[col].count())

slength
slength
0    45
1    50
2    43
3    12
Name: slength, dtype: int64
swidth
swidth
0    24
1    84
2    36
3     6
Name: swidth, dtype: int64
plength
plength
0    50
1    11
2    61
3    28
Name: plength, dtype: int64
pwidth
pwidth
0    50
1    28
2    43
3    29
Name: pwidth, dtype: int64


### EWD - Scenario 2: k = 7

In [45]:
# Perform discretization
k = 7
start = time.time() # Starting  time
data_ewd2 = ewd_disc(data, k)
end = time.time()
ewd_t = end - start
print("Discretization time, EWD, k = ", k,":",ewd_t) # Total time execution

Discretization time, EWD, k =  7 : 0.01569199562072754


In [46]:
# OUTPUT:
data_ewd2.head()

,slength,swidth,plength,pwidth,label
0,1,4,0,0,0
1,1,2,0,0,0
2,0,3,0,0,0
3,0,3,0,0,0
4,1,4,0,0,0


In [47]:
## OUTPUT: Check number of instance in each interval in the data_ewd
# With equal width discretisation, each bin does not necessarily 
# contain the same number of observations.
for col in num_col:
    print(col)
    print(data_ewd2.groupby(col)[col].count())

slength
slength
0    16
1    30
2    34
3    28
4    25
5    10
6     7
Name: slength, dtype: int64
swidth
swidth
0     8
1    16
2    59
3    31
4    24
5     9
6     3
Name: swidth, dtype: int64
plength
plength
0    48
1     2
2     5
3    20
4    43
5    23
6     9
Name: plength, dtype: int64
pwidth
pwidth
0    48
1     2
2    10
3    26
4    30
5    17
6    17
Name: pwidth, dtype: int64


### EWD - Scenario 3: k = 10

In [48]:
# Perform discretization
k = 10
start = time.time() # Starting time
data_ewd3 = ewd_disc(data, k)
end = time.time()
ewd_t = end - start
print("Discretization time, EWD, k = ", k,":",ewd_t) # Total time execution

Discretization time, EWD, k =  10 : 0.016379833221435547


In [49]:
# OUTPUT:
data_ewd3.head()

,slength,swidth,plength,pwidth,label
0,2,6,0,0,0
1,1,4,0,0,0
2,1,4,0,0,0
3,0,4,0,0,0
4,1,6,0,0,0


In [50]:
## OUTPUT: Check number of instance in each interval in the data_ewd
# With equal width discretisation, each bin does not necessarily 
# contain the same number of observations.
for col in num_col:
    print(col)
    print(data_ewd3.groupby(col)[col].count())

slength
slength
0     9
1    23
2    14
3    27
4    22
5    20
6    18
7     6
8     5
9     6
Name: slength, dtype: int64
swidth
swidth
0     4
1     7
2    22
3    24
4    51
5    18
6     9
7    11
8     2
9     2
Name: swidth, dtype: int64
plength
plength
0    37
1    13
3     3
4     8
5    26
6    29
7    18
8    11
9     5
Name: plength, dtype: int64
pwidth
pwidth
0    41
1     8
2     1
3     7
4    21
5    20
6     6
7    23
8     9
9    14
Name: pwidth, dtype: int64


## Equal Frequency Discretization - EFD
- Reference: https://nbviewer.org/github/feature-engine/feature-engine-examples/blob/main/discretisation/EqualFrequencyDiscretiser.ipynb
- Parameter:
- q : int, default=10
    Desired number of equal frequency intervals / bins. In other words the
    number of quantiles in which the variables should be divided.

- variables : list
    The list of numerical variables that will be discretised. If None, the
    EqualFrequencyDiscretiser() will select all numerical variables.

- return_object : bool, default=False
    Whether the numbers in the discrete variable should be returned as
    numeric or as object. The decision is made by the user based on
    whether they would like to proceed the engineering of the variable as
    if it was numerical or categorical.

- return_boundaries: bool, default=False
    whether the output should be the interval boundaries. If True, it returns
    the interval boundaries. If False, it returns integers.

In [51]:
def efd_disc(data, k):
    ## set up the discretisation transformer
    efd_disc = efd(q=k, variables=num_col)
    ## fit the transformer
    efd_disc.fit(data)
    ## transform the data
    data_efd = efd_disc.transform(data)
    ## binner_dict_ stores the interval limits identified for each variable.
    efd_disc.binner_dict_
    return data_efd

### Define function efd_disc, inputs include dataset, number of intervals (k)

### EFD - Scenario 1: k = 4

In [52]:
# Perform discretization
k = 4
start = time.time() # Starting time
data_efd1 = efd_disc(data, k)
end = time.time()
efd_t = end - start
print("Discretization time, EFD, k = ", k,":", efd_t) # Total time execution

Discretization time, EFD, k =  4 : 0.016688108444213867


In [53]:
## OUTPUT: Check number of instance in each interval 
for col in num_col:
    print(col)
    print(data_efd1.groupby(col)[col].count())

slength
slength
0    41
1    39
2    35
3    35
Name: slength, dtype: int64
swidth
swidth
0    47
1    36
2    31
3    36
Name: swidth, dtype: int64
plength
plength
0    44
1    31
2    41
3    34
Name: plength, dtype: int64
pwidth
pwidth
0    41
1    37
2    38
3    34
Name: pwidth, dtype: int64


### EFD - Scenario 2: k = 7

In [54]:
# Perform discretization
k = 7
start = time.time() # Starting time
data_efd2 = efd_disc(data, k)
end = time.time()
efd_t = end - start
print("Discretization time, EFD, k = ", k,":",efd_t) # Total time execution

Discretization time, EFD, k =  7 : 0.0239412784576416


In [55]:
## OUTPUT
data_efd2.info()
## OUTPUT: Check number of instance in each interval in the data_efd
for col in num_col:
    print(col)
    print(data_efd2.groupby(col)[col].count())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   slength  150 non-null    int64
 1   swidth   150 non-null    int64
 2   plength  150 non-null    int64
 3   pwidth   150 non-null    int64
 4   label    150 non-null    int64
dtypes: int64(5)
memory usage: 6.0 KB
slength
slength
0    22
1    23
2    20
3    24
4    19
5    22
6    20
Name: slength, dtype: int64
swidth
swidth
0    24
1    23
2    36
3    12
4    13
5    24
6    18
Name: swidth, dtype: int64
plength
plength
0    23
1    21
2    22
3    21
4    21
5    23
6    19
Name: plength, dtype: int64
pwidth
pwidth
0    34
1    14
2    17
3    21
4    30
5    17
6    17
Name: pwidth, dtype: int64


### Scenario 3: k = 10

In [56]:
# Perform discretization
k = 10
start = time.time() # Starting time
data_efd3 = efd_disc(data, k)
end = time.time()
efd_t = end - start
print("Discretization time, EFD, k = ", k,":",efd_t) # Total time execution

Discretization time, EFD, k =  10 : 0.01904010772705078


In [57]:
## OUTPUT
data_efd3.info()
## OUTPUT: Check number of instance in each interval in the data_efd
for col in num_col:
    print(col)
    print(data_efd3.groupby(col)[col].count())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   slength  150 non-null    int64
 1   swidth   150 non-null    int64
 2   plength  150 non-null    int64
 3   pwidth   150 non-null    int64
 4   label    150 non-null    int64
dtypes: int64(5)
memory usage: 6.0 KB
slength
slength
0    16
1    16
2    13
3    20
4    15
5    15
6    13
7    12
8    17
9    13
Name: slength, dtype: int64
swidth
swidth
0    19
1    14
2    14
3    36
4    12
5    13
6    18
7     9
8    15
Name: swidth, dtype: int64
plength
plength
0    23
1    14
2    11
3    13
4    14
5    15
6    18
7    12
8    17
9    13
Name: plength, dtype: int64
pwidth
pwidth
0    34
1    14
2    12
3    18
4    20
5    18
6     5
7    15
8    14
Name: pwidth, dtype: int64


## Fixed Frequency Discretization - FFD

### Define function ffd_disc: modify input of function efd
Input include dataset, interval frequency (m)

In [58]:
def ffd_disc(data, m): # 
    n = len(data)
    ## set up the discretisation transformer
    ffd_disc = efd(q=round(n/m), variables=num_col) # number of bins = n/m
    ## fit the transformer
    ffd_disc.fit(data)
    ## transform the data
    data_ffd = ffd_disc.transform(data)
    ## binner_dict_ stores the interval limits identified for each variable.
    ffd_disc.binner_dict_
    return data_ffd

### FFD - Scenario 1: m = 10

In [59]:
# Perform discretization
m = 10
start = time.time() # Starting time
data_ffd1 = ffd_disc(data, m)
end = time.time()
ffd_t = end - start
print("Discretization time, FFD,  m = ", m, ":", ffd_t) # Total time execution

Discretization time, FFD,  m =  10 : 0.019372224807739258


In [60]:
## OUTPUT
data_ffd1.info()
## OUTPUT: Check number of instance in each interval
for col in num_col:
    print(col)
    print(data_ffd1.groupby(col)[col].count())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   slength  150 non-null    int64
 1   swidth   150 non-null    int64
 2   plength  150 non-null    int64
 3   pwidth   150 non-null    int64
 4   label    150 non-null    int64
dtypes: int64(5)
memory usage: 6.0 KB
slength
slength
0     11
1     11
2     10
3      9
4     11
5     13
6      8
7      7
8     15
9     13
10     7
11     5
12    10
13    12
14     8
Name: slength, dtype: int64
swidth
swidth
0     11
1     13
2      9
3     14
4     10
5     26
6     12
7     13
8      6
9     12
10     6
11    12
12     6
Name: swidth, dtype: int64
plength
plength
0     11
1     12
2     14
3      7
4      6
5     11
6     12
7     14
8      3
9     14
10    12
11     4
12    11
13    10
14     9
Name: plength, dtype: int64
pwidth
pwidth
0     34
1      7
2      9
3     10
4     18
5      8
6     12
7      4
8   

### FFD - Scenario 1: m = 30

In [61]:
# Perform discretization
m = 30
start = time.time() # Starting time
data_ffd2 = ffd_disc(data, m)
end = time.time()
ffd_t = end - start
print("Discretization time, EFD, m = ", m, ":", ffd_t) # Total time execution

Discretization time, EFD, m =  30 : 0.017087936401367188


In [62]:
## OUTPUT
data_ffd2.info()
## OUTPUT: Check number of instance in each interval
for col in num_col:
    print(col)
    print(data_ffd2.groupby(col)[col].count())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   slength  150 non-null    int64
 1   swidth   150 non-null    int64
 2   plength  150 non-null    int64
 3   pwidth   150 non-null    int64
 4   label    150 non-null    int64
dtypes: int64(5)
memory usage: 6.0 KB
slength
slength
0    32
1    33
2    30
3    25
4    30
Name: slength, dtype: int64
swidth
swidth
0    33
1    50
2    12
3    31
4    24
Name: swidth, dtype: int64
plength
plength
0    37
1    24
2    29
3    30
4    30
Name: plength, dtype: int64
pwidth
pwidth
0    34
1    26
2    38
3    23
4    29
Name: pwidth, dtype: int64


### FFD - Scenario 3: m = 60

In [63]:
# Perform discretization
m = 60
start = time.time() # Starting time
data_ffd3 = ffd_disc(data, m)
end = time.time()
ffd_t = end - start
print("Discretization time, FFD, m = ", m, ":", ffd_t) # Total time execution

Discretization time, FFD, m =  60 : 0.01614212989807129


In [64]:
## OUTPUT
data_ffd3.info()
## OUTPUT: Check number of instance in each interval
for col in num_col:
    print(col)
    print(data_ffd3.groupby(col)[col].count())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   slength  150 non-null    int64
 1   swidth   150 non-null    int64
 2   plength  150 non-null    int64
 3   pwidth   150 non-null    int64
 4   label    150 non-null    int64
dtypes: int64(5)
memory usage: 6.0 KB
slength
slength
0    80
1    70
Name: slength, dtype: int64
swidth
swidth
0    83
1    67
Name: swidth, dtype: int64
plength
plength
0    75
1    75
Name: plength, dtype: int64
pwidth
pwidth
0    78
1    72
Name: pwidth, dtype: int64


#### FFD, m = 100

In [65]:
# Perform discretization
m = 100
start = time.time() # Starting time
data_ffd4 = ffd_disc(data, m)
end = time.time()
ffd_t = end - start
print("Discretization time, FFD, m = ", m, ":", ffd_t) # Total time execution

Discretization time, FFD, m =  100 : 0.016482114791870117


In [66]:
## OUTPUT
data_ffd4.info()

## OUTPUT: Check number of instance in each interval
for col in num_col:
    print(col)
    print(data_ffd4.groupby(col)[col].count())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   slength  150 non-null    int64
 1   swidth   150 non-null    int64
 2   plength  150 non-null    int64
 3   pwidth   150 non-null    int64
 4   label    150 non-null    int64
dtypes: int64(5)
memory usage: 6.0 KB
slength
slength
0    80
1    70
Name: slength, dtype: int64
swidth
swidth
0    83
1    67
Name: swidth, dtype: int64
plength
plength
0    75
1    75
Name: plength, dtype: int64
pwidth
pwidth
0    78
1    72
Name: pwidth, dtype: int64


### Export discretized datasets

In [67]:
# EWD datasets:
data_ewd1.to_csv('iris_ewd1.csv', index = False) # k=4
data_ewd2.to_csv('iris_ewd2.csv', index = False) # k=7
data_ewd3.to_csv('iris_ewd3.csv', index = False) # k=10

In [68]:
# EFD datasets:
data_efd1.to_csv('iris_efd1.csv', index = False) # k=4
data_efd2.to_csv('iris_efd2.csv', index = False) # k=7
data_efd3.to_csv('iris_efd3.csv', index = False) # k=10


In [69]:
# FFD datasets:
data_ffd1.to_csv('iris_ffd1.csv', index = False) # m=10
data_ffd2.to_csv('iris_ffd2.csv', index = False) # m=30
data_ffd3.to_csv('iris_ffd3.csv', index = False) # m=60
data_ffd4.to_csv('iris_ffd4.csv', index = False) # m=100